# Progress Query
## 200 - PP
```sql
SELECT total, open, CONCAT(ROUND(100 * open / total, 2), "%") as "open (rel)", running, CONCAT(ROUND(100 * running / total, 2), "%") as "running (rel)", finished, CONCAT(ROUND(100 * finished / total, 2), "%") as "finished (rel)", successful, failed, CONCAT(ROUND(100 * successful / (successful + failed), 2), "%") as "success rate", CONCAT(ROUND(avgRuntimeFinished), "s") as "Average Time of Finished", CONCAT(ROUND(avgRuntimeFinished * open / running), "s") as "ETA" FROM (SELECT "aux" as pk, COUNT(*) as "open" FROM `lccv_200_pp` WHERE time_started is null) as t1 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as "running" FROM `lccv_200_pp` WHERE time_started is not null and time_end is null) as t2 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as finished, AVG(TIMESTAMPDIFF(SECOND, time_started, time_end)) as avgRuntimeFinished  FROM `lccv_200_pp` WHERE time_started is not null and time_end is not null) as t3 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as successful FROM `lccv_200_pp` where time_end is not null and exception is null) as t4 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as failed FROM `lccv_200_pp` where exception is not null) as t5 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as total FROM `lccv_200_pp`) as t6
```

## 200 - NoPP
```sql
SELECT total, open, CONCAT(ROUND(100 * open / total, 2), "%") as "open (rel)", running, CONCAT(ROUND(100 * running / total, 2), "%") as "running (rel)", finished, CONCAT(ROUND(100 * finished / total, 2), "%") as "finished (rel)", successful, failed, CONCAT(ROUND(100 * successful / (successful + failed), 2), "%") as "success rate", CONCAT(ROUND(avgRuntimeFinished), "s") as "Average Time of Finished", CONCAT(ROUND(avgRuntimeFinished * open / running), "s") as "ETA" FROM (SELECT "aux" as pk, COUNT(*) as "open" FROM `lccv_200_nopp` WHERE time_started is null) as t1 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as "running" FROM `lccv_200_nopp` WHERE time_started is not null and time_end is null) as t2 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as finished, AVG(TIMESTAMPDIFF(SECOND, time_started, time_end)) as avgRuntimeFinished  FROM `lccv_200_nopp` WHERE time_started is not null and time_end is not null) as t3 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as successful FROM `lccv_200_nopp` where time_end is not null and exception is null) as t4 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as failed FROM `lccv_200_nopp` where exception is not null) as t5 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as total FROM `lccv_200_nopp`) as t6
```

# Failed Runs
```sql
SELECT *  FROM `lccv_200_pp` WHERE `exception` IS NOT NULL and openmlid not in(3, 31, 188, 1049, 1067, 1111, 1461, 1486, 1590, 4135, 4534, 4541, 40668, 40670, 40978, 41143, 41147, 41150, 42733)  
ORDER BY `lccv_200_pp`.`openmlid` ASC
```

# Ordinary Result List
## 200 - PP
```sql
SELECT openmlid, algorithm, round(avg(errorrate), 2), avg(runtime), count(*) FROM `lccv_200_pp` where time_end is not null and exception is null and algorithm in ("5cv", "80lccv-flex") group by openmlid, algorithm 
```

## 200 - NoPP
```sql
SELECT openmlid, algorithm, round(avg(errorrate), 2), avg(runtime), count(*) FROM `lccv_200_nopp` where time_end is not null and exception is null and algorithm in ("5cv", "80lccv") group by openmlid, algorithm 
```

#  High LCCV Deviations
## 200 - PP
```sql
SELECT * FROM
(SELECT openmlid, seed, errorrate as err_5cv, runtime as runtime_5cv, executor as exec_5cv FROM `lccv_200_pp` where time_end is not null and exception is null and algorithm = "5cv") as t1
NATURAL JOIN
(SELECT openmlid, seed, errorrate as err_80lccvflex, runtime as runtime_80lccvflex, executor as exec_80lccvflex FROM `lccv_200_pp` where time_end is not null and exception is null and algorithm = "80lccv-flex") as t3
where runtime_5cv < runtime_80lccvflex

```
## 200 - NoPP
```sql
SELECT * FROM
(SELECT openmlid, seed, errorrate as err_5cv, executor as exec_5cv FROM `lccv_200_nopp` where time_end is not null and exception is null and algorithm = "5cv") as t1
NATURAL JOIN
(SELECT openmlid, seed, errorrate as err_80lccv, executor as exec_80lccv FROM `lccv_200_nopp` where time_end is not null and exception is null and algorithm = "80lccv") as t2
where abs(err_5cv - err_80lccv) >= 0.01
```

# Clean Failed Experiments
```sql
UPDATE `lccv` set host = null, executor = null, time_started = null, exception = null, time_end = null WHERE time_end is not null and exception is not null
```